## Data overview:
You are provided with unlabeled data for which labeling is to be computed. There are 20 subjects of fMRI scan in total, for each subject there are 16 (2x2x2x2) representations corresponding to – two different brain atlas partitions (Brainnetome and Schaefer200), times two different smoothing strategies, times two segments of scan sequence, times two different sequences. Shape structure for the dataset translates aas follows `[20*16 objects, 10 timesteps from scan sequence, 246 number of features in larger atlas]`. Note that since two atlases with different number of partitions are used, some data arrays are padded with `np.nan`s, so that data shape is uniform.

*Public Test Data:*

- IHB dataset: 10 subject

*Private Test Data:*

- IHB dataset: 10 subjects

File with all $20 \times 16$ scan data representations is available via this repository lfs. 

## Objective
The task aims to simulate a realistic research workflow:

-	Data Collection Constraints: Collecting fMRI data is challenging and costly, resulting in small proprietary datasets.
-	Dataset Variability: These open datasets inherently differ from proprietary data in aspects such as scanner type, geographic location of data collection, and average age of participants. Such aspect are simulated through choice of different brain states, time sequences, atlas-based aggregations.
The primary goal is to develop a model capable of identifying a person using fMRI as a "fingerprint" which is consistent across different perturbations and aspects of data.

## Performance Metric and Deliverables

*Evaluation Metric:* Adjusted Rand Score (rescaled to be between 0.0 for random prediction and worse and 1.0 for perfect labeling: $\text{ari}(y, \hat{y}) = \frac{\text{ri}(y, \hat{y}) - 0.85}{0.15}$)

*Required Deliverables:*
-	`<name>.csv` submission file that contains column named `prediction` which has same integer labels for objects corresponding to same class (subject's fMRI scan).

## Example

In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd

C:\Users\Public\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
common_brain_region_data

array([[-1.20805163e+00, -9.15837054e-01, -5.82438250e-01, ...,
         5.99499068e-01, -2.05547329e-01,  7.86760412e-01],
       [ 1.02389546e+00, -9.29716743e-01, -3.41629252e-01, ...,
         1.04673451e+00,  1.39276165e-01, -1.36538101e+00],
       [ 8.17854450e-01, -3.39151069e-01,  1.16818813e+00, ...,
        -1.25069185e-01,  1.50202891e-03,  4.01659570e-01],
       ...,
       [ 1.68166144e+00,  2.62376247e-01,  8.37748891e-02, ...,
        -3.42288649e-01, -5.66129102e-01, -7.67054871e-01],
       [-5.40128141e-01, -3.31080917e-01, -1.23031036e+00, ...,
        -5.28072129e-01,  6.94622447e-01,  3.66038916e-01],
       [-1.20782776e-01,  4.89435880e-01, -2.32544953e+00, ...,
        -8.99009184e-01,  9.17174650e-01,  1.58648053e+00]])

**Use standard scaling to account for different scan smoothing strategies, kmeans for computing cluster centers and subsequent distance-based ranking**

In [3]:
data = np.load('../data/ts_cut/ihb.npy')

common_brain_region_data = data[:, :, 0]

model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=20)
)

model.fit(common_brain_region_data)

C:\Users\Public\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


AttributeError: 'NoneType' object has no attribute 'split'

**Use greedy algorithm that takes top 16 objects that are closest to cluster centers and consumes them iteratively**

In [ ]:
cluster_distances = model.transform(common_brain_region_data)

labeling = np.zeros(len(data), dtype=int)
leftover_indexes = np.arange(len(data))
for i in range(20):
    distances_from_current_cluster_center = cluster_distances[:, i]
    if len(distances_from_current_cluster_center) > 16:
        top16 = np.argpartition(distances_from_current_cluster_center, 16)[:16]
        labeling[leftover_indexes[top16]] = i
        cluster_distances = np.delete(cluster_distances, top16, axis=0)
        leftover_indexes = np.delete(leftover_indexes, top16)
    else:
        labeling[leftover_indexes] = i

**form final submission**

In [ ]:
pd.DataFrame({'prediction': labeling}).to_csv('../submission.csv', index=False)